# Total Covariance 

This notebook demonstrates how to calculate the covariance of a dataset given covariances and means of two or more subsets of this dataset.  This technique can be used for parallelizing the computation or finding the covariance of a large dataset that cannot fit in memory.  

## Overview
Given two datasets X and Y of the same length N we can calculate their covariance using the following formula

$$Cov(X, Y) = \sum((X_i - \overline{X}) (Y_i - \overline{Y}) $$

Given two data sub sets A and B with two variables (x) and (y), their means $\mu_a$ and $\mu_b$, and covariances between between (x) and (y) of each individual data set as $C_a$ and $C_b$ we can calculate the total covariance $C_t$ of the whole dataset using the following formula.

$$C_t = \frac{(C_a (n_a-1)) + (C_b  (n_b-1)) + (\mu_a^{(x)} - \mu)  (\mu_a^{(y)} - \mu)  n_a  + (\mu_b^{(x)} - \mu) (\mu_b^{(y)} - \mu) n_b }{n-1}$$

For this tutorial we are going to use the famous iris dataset, and in the first part we only use the first two features (columns)

## Covariance between two variables

In [76]:
from sklearn import datasets
import numpy as np
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
X.shape

(150, 2)

First we compute the covariance of the two variables of the whole dataset.

In [77]:
np.cov(X, rowvar=False)

array([[ 0.68569351, -0.03926846],
       [-0.03926846,  0.18800403]])

The numpy produced a matrix of covariances in which the entries in the main diagonal are variances of variables, a thee covariance between two variables is in the positions (0,1) and (1,0)

In [78]:
cov_X = np.cov(X, rowvar=False)[0][1]
cov_X

-0.039268456375838923

The covariance between two variables is -0.03926846. 
We divide the dataset X in two subsets of sizes 60 and 90. 

In [79]:
n_a = 60
n_b = 150 - n_a

In [80]:
X_A = X[0:n_a,:]
X_B = X[n_a: ,:]

We extract covariances $C_a$ and $C_b$ of both subsets X_A and X_B

In [81]:
cov_a = np.cov(X_A, rowvar=False)[0][1]
cov_b = np.cov(X_B, rowvar=False)[0][1]
cov_a, cov_b

(0.028112994350282486, 0.11629213483146071)

We also compute the means of $X_a$ and $X_b$ along columns

In [82]:
mean_a = X_A.mean(axis=0)
mean_b = X_B.mean(axis=0)

We recover the grand mean using the following formula
$$\mu_g = \frac{\mu_a * n_a + \mu_b * n_b}{n_a + n_b}$$

In [83]:
grand_mean = (mean_a * n_a + mean_b * n_b) / (n_a + n_b)

Finally we calculate the covariance of the whole dataset from covariances of individual groups using the formula given at the beginning of this tutorial

In [84]:
grand_cov = ((cov_a * (n_a - 1)) + (cov_b * (n_b-1)) + \
    ((mean_a[0] - grand_mean[0]) * (mean_a[1] - grand_mean[1]) * n_a) + \
    ((mean_b[0] - grand_mean[0]) * (mean_b[1] - grand_mean[1]) * n_b)) / (n_a + n_b - 1) 
grand_cov

-0.039268456375838957

In [85]:
cov_X - grand_cov

3.4694469519536142e-17

The difference is very small, so we conclude we computed the covariance correctly.

We can also vectorize the computation and compute the covariance of all four variables available in the iris dataset. 

In [86]:
X = iris.data 
X_A = X[0:n_a,   :]
X_B = X[n_a:, :]
cov_a = np.cov(X_A, rowvar=False)
cov_b = np.cov(X_B, rowvar=False)
mean_a = X_A.mean(axis=0)
mean_b = X_B.mean(axis=0)
mean_x = (mean_a * n_a + mean_b * n_b) / (n_a + n_b)
grand_cov = (cov_a * (n_a - 1) + cov_b * (n_b-1) +\
             (np.outer((mean_a - mean_x), (mean_a - mean_x)) * n_a) +
             (np.outer((mean_b - mean_x), (mean_b - mean_x) * n_b))) / (n_a + n_b - 1)

In [87]:
grand_cov

array([[ 0.68569351, -0.03926846,  1.27368233,  0.5169038 ],
       [-0.03926846,  0.18800403, -0.32171275, -0.11798121],
       [ 1.27368233, -0.32171275,  3.11317942,  1.29638747],
       [ 0.5169038 , -0.11798121,  1.29638747,  0.58241432]])

We can also generalize the computation of the total covariance matrix from covariances and means of individual groups. Concretely, given $m$ sets of two variables (x) and (y) of size $n_i$, the means $\mu_i$ and the covariances $Cov_i(x,y)$ we calculate the total covariance

$$Cov_t(x,y) = \frac{\sum_{i=1}^{m} {(Cov_i(x,y) * (n_i-1)} + \sum_{i=1}^{m} {n_i(\mu_i^{(x)} - \mu^{(x)})  (\mu_i^{(y)} - \mu^{(y)}) }}{{N-1}}$$

In [88]:
def total_covariance(covs, means, N):
    c_shape = covs.shape
    grand_mean = np.dot(means.T, N) / (np.sum(N))
    ess = np.sum((covs.reshape((covs.shape[0], -1)) * (N - 1).reshape(N.shape[0], 1)).reshape(c_shape), axis=0)
    tgss = np.sum([ np.outer(x, x) * n for x, n  in zip(means - grand_mean, N)], axis=0) 
    return (ess + tgss) / (np.sum(N) - 1)

Let's split the iris data set in three subsets of unequal sizes and calculate the covariance matrix using the function we defined.

In [89]:
N1 = np.array([20, 70, 60])
groups = [ X[0:N1[0],:], X[N1[0]:N1[0] + N1[1],:], X[N1[0]+N1[1]:,:]]
covs = np.array([np.cov(g, rowvar=False) for g in groups])
means = np.array([g.mean(axis=0) for g in groups])
total_covariance(covs, means, N1)

array([[ 0.68569351, -0.03926846,  1.27368233,  0.5169038 ],
       [-0.03926846,  0.18800403, -0.32171275, -0.11798121],
       [ 1.27368233, -0.32171275,  3.11317942,  1.29638747],
       [ 0.5169038 , -0.11798121,  1.29638747,  0.58241432]])

We can verify the calculation by calculating covariance of the whole dataset

In [90]:
np.cov(X, rowvar=False) 

array([[ 0.68569351, -0.03926846,  1.27368233,  0.5169038 ],
       [-0.03926846,  0.18800403, -0.32171275, -0.11798121],
       [ 1.27368233, -0.32171275,  3.11317942,  1.29638747],
       [ 0.5169038 , -0.11798121,  1.29638747,  0.58241432]])

In [91]:
grand_covariance(covs, means, N1) - np.cov(X, rowvar=False) 

array([[ -1.22124533e-15,  -1.04083409e-16,  -2.22044605e-15,
         -1.22124533e-15],
       [ -1.04083409e-16,   5.55111512e-17,  -3.88578059e-16,
         -1.94289029e-16],
       [ -2.22044605e-15,  -3.88578059e-16,   2.22044605e-15,
         -8.88178420e-16],
       [ -1.22124533e-15,  -1.94289029e-16,  -8.88178420e-16,
         -3.33066907e-16]])